I played HackTM CTF 2023 with Social Engineering Experts, and we came in 6th!

![scoreboard](scoreboard.png)

We solved three out of five crypto challenges; the remaining two were not solved by any teams. The three challenges are:
* **d-phi-enc** (383 points)
* **kaitenzushi** (490 points / first blood)
* **broken oracle** (499 points)

# d-phi-enc

```
In CTF, there are many people who mistakenly encrypt p, q in RSA.
But this time...
```

This was a standard RSA-with-hint challenge with $e=3$, and the hint we were provided was the encrypted values of $d$ and $\phi$, i.e.

$$enc_d = d^3\bmod{n} \text{ and } enc_\phi = \phi^3 \bmod{n}.$$

We know that $de \equiv 1\pmod{\phi}$, so that $de=k\phi+1$ with $k$ being either 1 or 2.

This gives us exactly what we need to do [Franklin-Reiter](https://en.wikipedia.org/wiki/Coppersmith%27s_attack#Franklin%E2%80%93Reiter_related-message_attack), that is we can get the solution by computing the GCD of the two polynomials $x^3-enc_d$ and $\left(\frac{3x-1}{k}\right)^3-enc_\phi$. We need to brute force over all possible values of $k$, but there's really only two possibilities here.

In [1]:
from sage.all import *
from Crypto.Util.number import long_to_bytes

n = 24476383567792760737445809443492789639532562013922247811020136923589010741644222420227206374197451638950771413340924096340837752043249937740661704552394497914758536695641625358888570907798672682231978378863166006326676708689766394246962358644899609302315269836924417613853084331305979037961661767481870702409724154783024602585993523452019004639755830872907936352210725695418551084182173371461071253191795891364697373409661909944972555863676405650352874457152520233049140800885827642997470620526948414532553390007363221770832301261733085022095468538192372251696747049088035108525038449982810535032819511871880097702167
enc_d = 23851971033205169724442925873736356542293022048328010529601922038597156073052741135967263406916098353904000351147783737673489182435902916159670398843992581022424040234578709904403027939686144718982884200573860698818686908312301218022582288691503272265090891919878763225922888973146019154932207221041956907361037238034826284737842344007626825211682868274941550017877866773242511532247005459314727939294024278155232050689062951137001487973659259356715242237299506824804517181218221923331473121877871094364766799442907255801213557820110837044140390668415470724167526835848871056818034641517677763554906855446709546993374
enc_phi = 3988439673093122433640268099760031932750589560901017694612294237734994528445711289776522094320029720250901589476622749396945875113134575148954745649956408698129211447217738399970996146231987508863215840103938468351716403487636203224224211948248426979344488189039912815110421219060901595845157989550626732212856972549465190609710288441075239289727079931558808667820980978069512061297536414547224423337930529183537834934423347408747058506318052591007082711258005394876388007279867425728777595263973387697391413008399180495885227570437439156801767814674612719688588210328293559385199717899996385433488332567823928840559
enc_flag = 24033688910716813631334059349597835978066437874275978149197947048266360284414281504254842680128144566593025304122689062491362078754654845221441355173479792783568043865858117683452266200159044180325485093879621270026569149364489793568633147270150444227384468763682612472279672856584861388549164193349969030657929104643396225271183660397476206979899360949458826408961911095994102002214251057409490674577323972717947269749817048145947578717519514253771112820567828846282185208033831611286468127988373756949337813132960947907670681901742312384117809682232325292812758263309998505244566881893895088185810009313758025764867

def poly_gcd(a, b): 
    return poly_gcd(b, a % b) if b else a.monic()

for k in [1, 2]:
    x = Zmod(n)['x'].gen()
    d = int(-poly_gcd(x**3-enc_d, ((3*x-1)/k)**3-enc_phi)[0])
    print(long_to_bytes(pow(enc_flag, d, n)))

b"\x19\xba\xa4\t\x95\xd0<Y8\x16\x06\x10\xf1\xba\x947\x02\xe0Ga]\x9e,=\xd1\xcb39\xb9{\xd9!\xae\x19\x99\x96\x89{I\xb5\x99\x19\xba\xdcD\xf9`\x1e\x03dWW\xea\xe6\x17>OPI\xae\xa0\x1a\xd4'\x8a\xa3\r\x9b/\x90zwX\xec\x00Y\xed\xedJ\x8fw\x13\xb5\x11\xdd\xd2\xc9\xc4&\xdc\x8c}P\xf3\xea)\x89w\x8e:A\xb2\xbf\x0b\xeb\xe2\xf7B\x93\xd3@\xb7t\x8f)\x88QjbY\x1f[A\x98O\xa8\xaa.\x11\xae\xe9\xe3\xcdv\x84\x92\x02\xf3\xad\xc3\x1ca\xedf\xd3\xbaB\x97\x91\xd0\xecv\xcd\x03\xbe\x8f\x87\x7f\xcd\xd5_\x93'\xd3\x06P@\r{?w\xe3\xd48W6\xb5\x9b'\xef\x89\xd1\xb6Q\x01q\xe6;*Y\r\xcdw\xecy\x80\xf1\xe1\x14\xca\xdb\xab\xd1\xc3a\xd16\x98\x06:\xd7Z\xb5\xb36E;\x14j\xf3\xd2v.\xfc\xfbX\xf6\x88\xf5O4O\x8f\xb1\x9f\xd5\x7f\x0e\x8c\\\x0c\x95\x82\xc0\xb3\xfa,\x8b\x9dS\x14\x9a,r\xdc\xe0"
b"HackTM{Have you warmed up? If not, I suggest you consider the case where e=65537, although I don't know if it's solvable. Why did I say that? Because I have to make this flag much longer to avoid solving it just by calculating the cubic root of enc_flag.}"

# kaitenzushi

```
also known as conveyor belt sushi
```

This was an RSA-like challenge where we not given the exponent $e$. We were also told that the modulus $n$ satisfied

$$n = x_1^2 + e y_1^2 = x_2^2 + e y_2^2,$$

and we basically had to recover all the values. What we were given though, was the vectors $(x_1, x_2)$ and $(y_1, y_2)$ after undergoing a rotation by some unknown angle $\theta$. Additionally, the floating points were truncated to 1337 bits, but this was enough for most intents and purposes.

We solve this in 3 steps:
1. Solve $e$ from the equation $\left(x_1^2+x_2^2\right)+e\left(y_1^2+y_2^2\right)=2n$, since vector norms are preserved by rotation.
2. Binary search $\theta$ to recover the original values of $x_1, x_2, y_1, y_2$. When $\theta=0$ we get $x^2+ey^2<n$, and when $\theta=1$ we get $x^2+ey^2>n$, so we can just keep halving the interval until we get $x^2+ey^2=n$.
3. Use the above values to find a factor of $n$. This follows from

$$e \equiv \frac{-x_1^2}{y_1^2} \equiv \frac{-x_2^2}{y_2^2} \pmod{n},$$

so that

$$\left(\frac{x_1}{y_1}+\frac{x_2}{y_2}\right)\left(\frac{x_1}{y_1}-\frac{x_2}{y_2}\right) \equiv 0 \pmod{n},$$,

and we basically hope that the two factors divide $p$ and $q$ respectively.

In [2]:
F = RealField(1337)
n = 990853953648382437503731888872568785013804329239290721076418541795771569507440261620612308640652961121590348037236708702361580700250705591203587939980126323233833431892076634892318387020242015741789265095380967467201291693288654956012435416445991341222221539511583706970342630678909437274145759598920314784293470918464283814408418704426938549136143925649863711450268227592032494660523680280136089617838412326902639568680941504799777445608524961048789627301462833
c = 312168688094168684887530746663711142224819184527420449851136749248641895825646649162310024737395663075921549510262779965673286770730468773215063305158197748549937395602308558217528064655976647148323981103647078862713773074121667862786737690376212246588956833193632937835958166526006128435536115531865213269197137648990987207140262543956087199861542889002996727146832659889656384027201202873352819689303456895088190857667281342371263570535523695457095802010885279
x = vector([F('9.93659400123277470926327676478883140697376509010297766512845139881487348637477791719517951397052010880811619509960535668814993293095146708957649613776125686226138447162258666762024346093786649228730054881453449071976210130217897905782845690384638460560301964009359233596889465133986468021963885911072779457835979983964294586954038412718305000570678333513135467257498071686562749882446495426493483289204e230'), F('-1.20540611958254673086539287012513674064476659427085664430224592760592531301348857885707154893714440960111029743010026152632150988429192286517249118913535366887447596463819555191858702861383725310592687577510708180057642425944345656558038998574368521689142109798891989865473206201635908814994474491537093810680632691594902962470061189337645818851446622588020765058461348047229165216450857822980873846637e230')])
y = vector([F('9.02899744041999015549480362358897037217795303901085937071039171882835297563545959015336648016772002396355451308252077767567617065937943765701645833054147976124287566465577491039263554806622908070370269238064956822205986576949383035741108310668397305286076364909407660314991847716094610949669608550117248147017329449889127749721988228613503029640191269319151291514601769696635252288607881829734506023770e191'), F('2.82245306887391321716872765000993510002376761684498801971981175059452895101888694909625866715259620501905532121092041448909218372087306882364769769589919830746245167403566884491547911250261820661981772195356239940907493773024918284094309809964348965190219508641693641202225028173892050377939993484981988687903270349415531065381420872722271855270893103191849754016799925873189392548972340802542077635974e192')])

In [3]:
# Step 1: Solve for e
e = int((2*n-x.norm()**2)/y.norm()**2)
f'{e=}'

'e=111578009802636409437123757591617048189760145423552421418627338749835916561801'

In [4]:
#Step 2: Solve for x1, x2, y1, y2
def get(theta):
    R = matrix(F, [[cos(theta), -sin(theta)], [sin(theta), cos(theta)]])
    xx = R * x
    yy = R * y
    xx = [int(z + F(1/2)) for z in xx]
    yy = [int(z + F(1/2)) for z in yy]
    t0 = xx[0]**2 + e*yy[0]**2
    t1 = xx[1]**2 + e*yy[1]**2
    return t1, xx, yy

lo, hi = F(0), F(1)
assert get(lo)[0]<n
assert get(hi)[0]>n
for _ in range(2000):
    mid = (lo+hi)/2
    g, xx, yy = get(mid)
    if g == n:
        break
    elif g > n:
        hi = mid
    else:
        lo = mid

x1, x2 = xx
y1, y2 = yy
assert x1**2 + e*y1**2 == n
assert x2**2 + e*y2**2 == n
x1, x2, y1, y2

(993315378106395196440156892634615357425859001976376351903878161126954317590016249318316631584063366449446002974804447367756266228508159317926113473123770241598131922105753478630709094061327843793983555725542453353312556415777678937,
 123343431936894440973263647479974540141395074556779828339916509613682879668610901423506961118285523166037774054833601787794419590891163752205158573276826154790166536984681500991748749778629881670438838666011425669518792357094873553,
 193518098174342694414424160720807163740044134017573004218248685165604434384710484681124817651698709818703976889508767807895216618103609127904817977547152172876909535027087606807328610207963608,
 2957028917590401838272414886210261099554152128524012256631787151968768935090286908219944634008304129914083074684507666539700290047827545862670465906725813971398170535104589598065683927537059268)

In [5]:
p = int(gcd(n, x1*y2-x2*y1))
q = n // p
assert 1 < p < n and n == p * q

d = pow(e, -1, (p-1)*(q-1))
long_to_bytes(pow(c, d, n) ^ x1 ^ x2 ^ y1 ^ y2)

b'HackTM{r07473_pr353rv35_50m37h1n6}'

# broken oracle

```
I have reimplemented a cryptosystem, but it sometimes behaves strangely. But I don't think it matters.
```

This was a fun novel cryptosystem, also not too dissimilar from RSA, and specified [here](https://www.cs.umd.edu/~gasarch/TOPICS/miscrypto/rabinwithrecip.pdf). Basically, your private key are two primes $p$ and $q$, and the public key is the modulus $n=pq$ and a random value $c$ that is a quadratic non-residue modulo both $p$ and $q$.

Encryption is performed on a message $M \in \mathbb{Z}_n$ by evaluating $r = M + \frac{c}{M} \bmod{n}$. Of course, since this is a quadratic, you have four possible messages $M$ encrypting to the same $r$, so you also provide auxiliary information $s \in \{1, -1\}$ and $t \in \{0,1\}$ to allow you to pick the correct $M$. We won't go into too much detail as to what $s$ and $t$ do, but the short of it there are two solutions for $M \bmod{p}$ and two solutions for $M \bmod{q}$.

This of course means that only one quarter of all $r$ values are possible, and in theory the decrypt procedure can detect this. However, the implementation uses a GCD-style root-finding that assumes a solution exists, so when it doesn't you get a nonsense term. For all intents and purposes, we can think of this decrypted value as random (but deterministic). The only important bit is that it knows a priori that that the roots of $M^2-rM+c$ must sum to $r$, so once it finds one root $x_0$ using the aforementioned GCD procedure, it determines the other root by taking $r-x_0$.

Now, for the CTF challenge we don't get an encryption or decryption protocol; just a composition of decrypt followed by encrypt. In addition, we don't know the public key or the private key, so we need to determine $p$, $q$, and $c$.

Our solution to this challenge then follows the following high-level idea:
1. We find a value of $r_0$ such that $M^2-r_0M+c$ has no roots modulo $p$ or $q$. This happens only if we get back four different values of $r$ when we ask the oracle to re-encrypt $(r_0,s,t)$ for all possible $s$ and $t$.
2. What happens in the re-encryption phase is we've constructed all four possible values of $M$, i.e. two roots mod p times two roots mod q. For each $M$, we get back the value $M + \frac{c}{M} \bmod{n}$. What this means is that we have two pairs of solutions that are congruent mod p, and two pairs of solutions that are congruent mod q. So we can find $p$ and $q$ by taking GCDs this way. Sometimes the GCD returns a small multiple of $p$ or $q$ and we can trial-factorise out small values, but we are lazy and just keep repeating until they're both prime.
3. Now that we have $p$ and $q$, all we need left is to find $c$. Here we use two messages $M_1$ and $M_2$ that are not congruent mod p or q (again, there are two such pairs), though we really only know the corresponding $r$ values and not the $M$s. This is fine, because we have a system of three simultaneous equations with three unknowns:

$$\begin{align*}M_1 + M_2 &= r_0,\\
M_1^2 - r_1M_1 + c &= 0,\\
M_2^2 - r_2M_2 + c &= 0.\end{align*}$$

We solve this using resultants (or algebra by hand, it really reduces down to a linear equation), and now we have all the variables to decrypt the flag!

In [6]:
from pwn import *
from math import gcd
from broken_oracle import Pubkey, Privkey, Enc, decrypt

#sh = process(['python', 'broken_oracle.py'])
sh = remote('34.141.16.87', 50001)

def readval():
    sh.readuntil(b'=')
    return int(sh.readline())

sh.readuntil(b'flag:')
enc = Enc(r=readval(), s=readval(), t=readval())
enc

[x] Opening connection to 34.141.16.87 on port 50001
[x] Opening connection to 34.141.16.87 on port 50001: Trying 34.141.16.87
[+] Opening connection to 34.141.16.87 on port 50001: Done


r = 1635498391299924303911835176366375116928352619114795234043981217574163192073146994768710439072448307161280630648337255782305843228870494842191648311478254777561047273075537658743064804734157737884463182964463434454026871752928674604311974346437712674135352068325114358197499320938914517031762352613654612421769616985836914868670018095322800180823865046711019597168344126871401938898276174508082275670818514481011170847251681954504301602327398311335847394359693692348459807262272474346370769572296810591231894458751260285114129991191532577624818094421305576164946579526196811235438354364815925115664174630014053070310
s = -1
t = 0

In [7]:
def reencrypt(r,s,t):
    sh.sendline(f'{r},{s},{t}'.encode())
    sh.readuntil(b's, t = ')
    reply = sh.readline()
    if b'wrong' in reply:
        return
    sh.readuntil(b'r = ')
    return int(sh.readline())

for r0 in range(1, 9999):
    rs = [reencrypt(r0,s,t) for s in [-1,1] for t in [0,1]]
    if None not in rs and len(set(rs)) == 4:
        gcds = sorted([
            gcd(rs[0]-rs[1], rs[2]-rs[3]),
            gcd(rs[0]-rs[2], rs[1]-rs[3]),
            gcd(rs[0]-rs[3], rs[1]-rs[2]),
        ])
        p, q = gcds[1:]
        if gcds[0] == 1 and is_prime(p) and is_prime(q):
            break
        
n = p * q
print(f'break at {r0=}')
f'{n=}'

break at r0=37


'n=13126522941067286887708018862234615983439242814638980480970091924404675319296651347079046877405823369022252203863416440213077005733414716900448195807460402871711902781240364930214536150637719648673022711800649492368564881838518833105853202457227937069799096937449923032085605163999708877830504465254604888804669781468087296561594222030779633707956852504683419804942959629196073692922217392552171017723062141649059459023670225289395751037913678531401798387681599449970355039594571078496173457644956832609449297372321904466725323070319898789456450831226037945220857676558804448300920896097349743358480016729188493682037'

In [8]:
r1 = mod(rs[0], n)
r2 = next(r for r in rs if (r-r1)%p and (r-r1)%q)

M1,M2,c = Zmod(n)['M1,M2,c'].gens()
c.parent()._has_singular = False

f0 = M1 + M2 - r0
f1 = M1**2 - r1*M1 + c
f2 = M2**2 - r2*M2 + c

res = f0.resultant(f1,M1).resultant(f2,M2)
c = int(-res.univariate_polynomial().monic()[0])
f'{c=}'

'c=12006777197869021488960385005085784061848024759413996349794183527223441181774934513793596843630301182637379916558689252273931991944751624710870798346338926910498418436695906804694308327284740483494712912392570629096823723982697520522994919768425830980655280374916133006500303693984089856607044912776130682662136559151757135967756700467295221964900336589257156519362491820396428260683356102245060360671103889348949268252415266633325885483764446192447184211034599502838942284411539860326766319970582475064841097081495258468124480098876821947592310657949115185933405262263594001119785437019016529382278997267171413642508'

In [9]:
long_to_bytes(decrypt(enc, Pubkey(n=n,c=c), Privkey(p=p,q=q)))

b'HackTM{h4v3_y0u_r34lly_f0und_4ll_7h3_bu65...?}go\x0e\xd9\xa9\xf6\xe8\xa5,u9\x14\xe1\xe8\x04\xc5\x07Y\xf8\xcc\xd5HC\xad\xf5\xf9\x83\xec\x85T\xbc\xce\xc3\xd8\xb3\x11|\xe4@$\xf45\x18\\\x10j_\x9e\x16\xc9=p\xb8+Y\xf0\xd0\x08_\xc20m\x1dmC\x99\xc1D\xb1b\x10\xe9)_,\xbf-[M$.\x8be\xeb\xdf:\x84\x86\x01\x84\xaf\xe4\x88_\x9e&\xed^LW5\xc17u\xb0\nk\xb9\xf8\xc0a\x93\x81\xb9\xb7\xa2/\x08\x1fb\x03j\xff\xd7\x17\x84\x06\xa3\x9f\xbbb\x8cTh<\x87-\xb6\xb8w\xba\xc94\xd1\xaf\x98$\xf2A4\xe9\xac\xe8\xfd\x87w_X\x8dV\xb9\x06\x9c\xe2\xc2\xa9\xd9\xac\xafu\xdc\xc0\xbe\x07\xfc[\xe3-\xd2\xf6|\xccgP\xbd"t5\x0bL)\x0bQy\x89We\xfe9N\xa4\xdd\x86@\x82\xfc\xbf\xa1\xec'